In [ ]:
import rebound
from astropy.time import Time
import numpy as np
from matplotlib import pyplot as plt

'''
This is a initialized sim with the planets and Sun in their positions at the 
launch time of Voyager 1, as taken from the JPL Horizons Database.
'''

def Tconvert(time):
    t = Time(time, format='jd', scale='tdb', out_subfmt='date_hm')
    return t.iso

def init(time, name):
    start_date = Tconvert(time)  # put in in the form required by Horizons ephemeris
    sim = rebound.Simulation()
    sim.units = ('days', 'AU', 'Msun')
    sim.add("Sun", date=start_date)
    sim.add("Mercury", date=start_date)
    sim.add("Venus",date=start_date)
    sim.add("399", date=start_date)
    sim.add("Mars", date=start_date)
    sim.add("Jupiter", date=start_date)
    sim.add("Saturn", date=start_date)
    sim.add("Uranus", date=start_date)
    sim.add("Neptune", date=start_date)
    sim.add("Pluto", date=start_date)
    sim.add("-31", date=start_date, m=3.629e-28) # this is Voyager 1, mass ref: http://nssdc.gsfc.nasa.gov/planetary/voyager.html
    sim.save("start{}.bin".format(name))

In [40]:
def get_day(jd, base_date):
    ''' get_day(float, float) --> float
    
    Tells you how many days from the start of the simulation a particular time is.
    Inputs are in Julian days, output is days from start of simulation.
    '''
    return jd - base_date

#run the line below to set up the sim
#init(float(time_data[0]), 'new_days') 

data_file = 'all_data_days.txt'
time_data = np.loadtxt(data_file, dtype='float', delimiter = ',', unpack=True, usecols=[0])
voy_final = np.loadtxt(data_file, dtype='float', delimiter = ',', skiprows=len(time_data)-1, unpack=True, usecols=[2,3,4])
times = np.arange(0., len(time_data)-1, 1.)


sim = rebound.Simulation.from_file("startnew_days.bin")
# Ensure sim parameters are what we want, set up particle array
sim.integrator = "ias15" # IAS15 is the default integrator, so we actually don't need this line
sim.move_to_com()        # We always move to the center of momentum frame before an integration
ps = sim.particles       # ps is now an array of pointers and will change as the simulation runs

c = 2294.884424601948 # days in (year*2*pi)
# Now add kicks as needed; form is ([x-kick, y-kick, z-kick]), units of au/(year*2pi)
kicks = [] 
kicks.append([5.84564348e-04/c ,-1.76732930e-05/c, -2.27558408e-04/c]) # @ before Jupiter
kicks.append([0.0002170668729999936/c, 0.001900560368000015/c,-3.1497304450000055e-05/c]) # to get simulated probe to match Voyager 1 @ Jupiter
kicks.append([-4.39985253/c, 2.98410966/c, 0.09549758/c]) # @ before Saturn
kicks.append([4.479490865833/c, 3.169782372502/c,  -0.2318683151379/c]) # to get simulated probe to match Voyager 1 @ Saturn
kicks.append([0.29341067568746676/c,  -0.12721212343561178/c,  0.39201543063530386/c]) # kick needed @ some point after saturn

# Add in the times at which the kicks should take place, in Julian days
kick_t = []
kick_t.append(get_day(2443397.5973599595, time_data[0])) # @ before Jupiter
kick_t.append(get_day(2443930.5720933327, time_data[0])) # @ Jupiter
kick_t.append(get_day(2443937.4743615612, time_data[0])) # @ before Saturn
kick_t.append(get_day(2444009.8291733307, time_data[0])) # @ Saturn
kick_t.append(get_day(2444018.6247764258, time_data[0])) # Post Saturn


def integrate(time_array, kick_times, kick_array):
    '''integrate(numpy array, numpy array, numpy array) --> sim.status, numpy array
    
    This function integrates the simiulation to the last time value provided in time_array.
    At each step, it checks to see whether it is at (or has exceeded) the time of the 
    velocity change. If so, it adds a velocity specified in kick_array to the probe
    (ps[10]). It then removes that time and the velocity from the respective arrays.
    '''
    k_out = []
    step = 1 # this will be incremented everytime a velocity change occurs
    progress = 0 # this exists just to give a sense of progress
    for time in time_array:
        if (len(kick_times) == 0):
            sim.integrate(time)
            progress += 1
            if progress % 1000 == 0:
                print "Now on day {}".format(sim.t)
                print 'delta x', ps[10].x - voy_final[0]
                print 'delta y', ps[10].y - voy_final[1]
                print 'delta z', ps[10].z - voy_final[2]
        elif time < kick_times[0]:
            sim.integrate(time)
            progress += 1
            if progress % 1000 == 0:
                print "Now on day {}".format(sim.t)
        else:
            print "Now on day {}".format(sim.t)
            print 'kick # {}'.format(step)
            step += 1 
            ax, ay, az = ps[10].vx, ps[10].vy, ps[10].vz
            sim.integrate(time)
            ps[10].vx += kick_array[0][0]
            ps[10].vy += kick_array[0][1]
            ps[10].vz += kick_array[0][2]
            ax = ps[10].vx - ax
            ay = ps[10].vy - ay
            az = ps[10].vz - az
            k_out.append([kick_times[0], [ax, ay, az]])  
            # instead of removing the elements, just redefine the arrays to avoid index out of range errors
            kick_times = kick_times[1:]
            kick_array = kick_array[1:]
            progress += 1
            if progress % 1000 == 0:
                print "Now on day {}".format(sim.t)
    # distance at end of sim needs to be implemented 
    #dist = np.sqrt((ps[10].x-voy_final[0])**2 + (ps[10].y-voy_final[1])**2 + (ps[10].z-voy_final[2])**2)# * 1.496e+8 to make it in km
    return sim.status(), k_out#, dist
    

In [41]:
sim.getWidget(size=(700,500),orbits=True)

In [42]:
integrate(times, kick_t, kicks)

Now on day 5.0
kick # 1
Now on day 538.0
kick # 2
Now on day 545.0
kick # 3
Now on day 617.0
kick # 4
Now on day 626.0
kick # 5
Now on day 999.0
delta x 20.6670981828
delta y 108.013119224
delta z -78.4252542614
Now on day 1999.0
delta x 15.3506284456
delta y 97.8145149106
delta z -78.2096121336
Now on day 2999.0
delta x 11.10617608
delta y 88.3555454153
delta z -78.026281997
Now on day 3999.0
delta x 7.16303943069
delta y 79.2441627138
delta z -77.8532372252
Now on day 4999.0
delta x 3.35862903614
delta y 70.3259713834
delta z -77.6852077275
Now on day 5999.0
delta x -0.36627792576
delta y 61.5308010168
delta z -77.5201561405
Now on day 6999.0
delta x -4.03968985921
delta y 52.8210173984
delta z -77.3570814263
Now on day 7999.0
delta x -7.67703198954
delta y 44.1740667206
delta z -77.1954168753
Now on day 8999.0
delta x -11.2876983513
delta y 35.5753473962
delta z -77.0348100682
Now on day 9999.0
delta x -14.8778315898
delta y 27.0148515343
delta z -76.8750266919
Now on day 10999.0
de

(None,
 [[5.3890266264788806,
   [-0.00028464861915653873, 4.674606755572519e-05, -8.272793810327507e-07]],
  [538.36375999962911,
   [-8.155747503255616e-06, 5.8783979554856685e-05, -1.7629622444359134e-06]],
  [545.26602822821587,
   [-0.0028110800278956915, 0.003066567379029911, 5.931160604028037e-06]],
  [617.62083999766037,
   [0.0019604186553779258, 0.0013750593522855493, -0.00010118804141443888]],
  [626.41644309274852,
   [0.0001363602453713659, -6.146435877439974e-05, 0.0001706706035065661]]])

In [43]:
print voy_final

[ -27.88790625 -108.60969675   78.60676397]


In [36]:
Tconvert(time_data[0])


'1977-09-05 17:00'

In [39]:
print kick_t
for i in [2443397.5973599595, 2443930.5720933327, 2443937.4743615612, 2444009.8291733307, 2444018.6247764258]:
    print Tconvert(i)
    

[5.3890266264788806, 538.36375999962911, 545.26602822821587, 617.62083999766037, 626.41644309274852]
1977-09-11 02:20
1979-02-26 01:43
1979-03-04 23:23
1979-05-16 07:54
1979-05-25 02:59
